# Testing the Al-Tadmuriyah Model

This notebook focuses on evaluating the performance of the final Al-Tadmuriyah model. After loading the best-performing model from the previous notebook, I test its ability to generate text and assess its performance across various scenarios.

The objective is to ensure that the model is capable of generating text that stays true to the unique linguistic style and thematic content of Shaykh al-Islam Ibn Taymiyyah’s Al-Tadmuriyah. Through these tests, I explore the model's strengths and limitations, providing insights into how well it handles different tasks related to text prediction and generation.

This notebook covers:

Loading and testing the fine-tuned Al-Tadmuriyah model.

Evaluating the model's ability to generate coherent and contextually relevant text.

Observations on the model's performance and areas for potential improvement.

In [1]:
! pip install -U transformers datasets peft
! pip install arabert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which i

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
from arabert.preprocess import ArabertPreprocessor
from peft import PeftModel

In [3]:
# Load model directly
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-large")
model = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-large", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

configuration_aragpt2.py:   0%|          | 0.00/11.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/aubmindlab/aragpt2-large:
- configuration_aragpt2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_aragpt2.py:   0%|          | 0.00/83.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/aubmindlab/aragpt2-large:
- modeling_aragpt2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/3.20G [00:00<?, ?B/s]

In [4]:
model = PeftModel.from_pretrained(model, 'ahmadAlrabghi/al_tadmoreyyah_model_public_adapter_1') # loading the adapter layer and apply it on the base model

adapter_config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/189M [00:00<?, ?B/s]

In [5]:
model.to('cuda') 

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): AraGPT2LMHeadModel(
      (transformer): AraGPT2Model(
        (wte): Embedding(64000, 1280)
        (wpe): Embedding(1024, 1280)
        (emb_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-35): 36 x AraGPT2Block(
            (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
            (attn): AraGPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D()
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1280, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=3840, bias=False)
                )
                (lora_embedding_A): ParameterDict()
    

In [6]:
tokenizer = AutoTokenizer.from_pretrained('aubmindlab/aragpt2-large', trust_remote_code=True)

# adding the pad token
tokenizer.add_special_tokens({'pad_token':'[PAD]'})
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

vocab.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:

gen_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer,
                        trust_remote_code=True)
dataset = load_dataset('ahmadAlrabghi/al_tadmoreyyah')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', '

README.md:   0%|          | 0.00/531 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/220k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/81 [00:00<?, ? examples/s]

In [8]:
text = dataset['train']['combined'][30]
text

'إلا الله والراسخون في العلم يقولون آمنا به كل من عند ربنا وما يذكر إلا أولو الألباب﴾ وجمهور سلف الأمة وخلفها على أن الوقف عند قوله ﴿وما يعلم تأويله إلا الله﴾ وهذا هو المأثور عن أبي بن كعب وابن مسعود وابن عباس وغيرهم وروي عن ابن عباس أنه قال التفسير على أربعة أوجه تفسير تعرفه العرب من كلامها وتفسير لا يعذر أحد بجهالته وتفسير يعلمه العلماء وتفسير لا يعلمه إلا الله من ادعى علمه فهو كاذب وقد روي عن مجاهد وطائفة أن الراسخين في العلم يعلمون تأويله وقد قال مجاهد عرضت المصحف على ابن عباس من فاتحته إلى خاتمته أقف عند كل آية وأسأله عن تفسيرها ولا منافاة بين القولين عند التحقيق فإن لفظ التأويل قد صار بتعدد الاصطلاحات مستعملا في ثلاثة معان أحدها وهو اصطلاح كثير من المتأخرين من المتكلمين في الفقه وأصوله أن التأويل هو صرف اللفظ عن الاحتمال الراجح إلى الاحتمال المرجوح لدليل يقترن به وهذا هو الذي عناه أكثر من تكلم من المتأخرين في تأويل نصوص الصفات وترك تأويلها وهل هذا محمود أو مذموم وحق أو باطل الثاني أن التأويل بمعنى التفسير وهذا هو الغالب على اصطلاح مفسري القرآن كما يقول ابن جرير وأمثاله من المصنفي

In [9]:
araba_prep = ArabertPreprocessor(model_name='aubmindlab/aragpt2-large')
cleaned_text = araba_prep.preprocess('وغيرهم وروي عن ابن عباس أنه قال التفسير على أربعة أوجه تفسير تعرفه العرب من كلامها وتفسير لا يعذر أحد بجهالته وتفسير يعلمه العلماء وتفسير لا يعلمه إلا الله من ادعى علمه فهو كاذب وقد روي عن مجاهد وطائفة أن الراسخين في العلم يعلمون تأويله وقد قال مجاهد عرضت المصحف على ابن عباس من فاتحته إلى خاتمته أقف عند كل آية وأسأله عن تفسيرها ولا منافاة بين القولين عند التحقيق فإن لفظ التأويل قد صار بتعدد الاصطلاحات مستعملا في ثلاثة معان أحدها وهو اصطلاح كثير من المتأخرين من المتكلمين في الفقه وأصوله')
cleaned_text

'وغيرهم وروي عن ابن عباس أنه قال التفسير على أربعة أوجه تفسير تعرفه العرب من كلامها وتفسير لا يعذر أحد بجهالته وتفسير يعلمه العلماء وتفسير لا يعلمه إلا الله من ادعى علمه فهو كاذب وقد روي عن مجاهد وطائفة أن الراسخين في العلم يعلمون تأويله وقد قال مجاهد عرضت المصحف على ابن عباس من فاتحته إلى خاتمته أقف عند كل آية وأسأله عن تفسيرها ولا منافاة بين القولين عند التحقيق فإن لفظ التأويل قد صار بتعدد الاصطلاحات مستعملا في ثلاثة معان أحدها وهو اصطلاح كثير من المتأخرين من المتكلمين في الفقه وأصوله'

In [10]:
dataset['train']['combined'][30]

'إلا الله والراسخون في العلم يقولون آمنا به كل من عند ربنا وما يذكر إلا أولو الألباب﴾ وجمهور سلف الأمة وخلفها على أن الوقف عند قوله ﴿وما يعلم تأويله إلا الله﴾ وهذا هو المأثور عن أبي بن كعب وابن مسعود وابن عباس وغيرهم وروي عن ابن عباس أنه قال التفسير على أربعة أوجه تفسير تعرفه العرب من كلامها وتفسير لا يعذر أحد بجهالته وتفسير يعلمه العلماء وتفسير لا يعلمه إلا الله من ادعى علمه فهو كاذب وقد روي عن مجاهد وطائفة أن الراسخين في العلم يعلمون تأويله وقد قال مجاهد عرضت المصحف على ابن عباس من فاتحته إلى خاتمته أقف عند كل آية وأسأله عن تفسيرها ولا منافاة بين القولين عند التحقيق فإن لفظ التأويل قد صار بتعدد الاصطلاحات مستعملا في ثلاثة معان أحدها وهو اصطلاح كثير من المتأخرين من المتكلمين في الفقه وأصوله أن التأويل هو صرف اللفظ عن الاحتمال الراجح إلى الاحتمال المرجوح لدليل يقترن به وهذا هو الذي عناه أكثر من تكلم من المتأخرين في تأويل نصوص الصفات وترك تأويلها وهل هذا محمود أو مذموم وحق أو باطل الثاني أن التأويل بمعنى التفسير وهذا هو الغالب على اصطلاح مفسري القرآن كما يقول ابن جرير وأمثاله من المصنفي

In [11]:
gen_pipeline(cleaned_text,
    num_beams=10,
    max_length=200,
    top_p=0.1,
    repetition_penalty = 3.0,
    no_repeat_ngram_size = 3,
    temperature=0.1
             )[0]['generated_text']

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'وغيرهم وروي عن ابن عباس أنه قال التفسير على أربعة أوجه تفسير تعرفه العرب من كلامها وتفسير لا يعذر أحد بجهالته وتفسير يعلمه العلماء وتفسير لا يعلمه إلا الله من ادعى علمه فهو كاذب وقد روي عن مجاهد وطائفة أن الراسخين في العلم يعلمون تأويله وقد قال مجاهد عرضت المصحف على ابن عباس من فاتحته إلى خاتمته أقف عند كل آية وأسأله عن تفسيرها ولا منافاة بين القولين عند التحقيق فإن لفظ التأويل قد صار بتعدد الاصطلاحات مستعملا في ثلاثة معان أحدها وهو اصطلاح كثير من المتأخرين من المتكلمين في الفقه وأصوله أن التأويل هو صرف اللفظ عن الاحتمال الراجح إلى الاحتمال المرجوح لدليل يقترن به وهذا هو الذي عناه أكثر من تكلم من المتأخرين في تأويل نصوص الصفات وترك تأويلها وهل هذا محمود أو مذموم وحق أو باطل الثاني أن التأويل بمعنى التفسير وهذا هو الغالب على اصطلاح مفسري القرآن كما يقول ابن جرير وأمثاله من المصنفين في التفسير واختلف علماء التأويل ومجاهد إمام المفسرين قال الثوري إذا جاءك التفسير عن مجاهد فحسبك به وعلى تفسيره يعتمد'